<a href="https://colab.research.google.com/github/boysbytes/colab-chatbot/blob/main/colab_gemma3_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Give access to your Google Drive

Allow the notebook to access your Google Drive files and set up the `ollama_models` directory.

This directory will contain the models you download later.

In [ ]:
!pip install gradio

import os
import threading
import subprocess
import time
import requests
import json
import gradio as gr

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create ollama_models folder if it doesn't exist
ollama_models_path = '/content/drive/MyDrive/ollama_models'
if not os.path.exists(ollama_models_path):
    os.makedirs(ollama_models_path)
    print(f"Created directory: {ollama_models_path}")
else:
    print(f"Directory already exists: {ollama_models_path}")

### Install dependencies

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

# Install libraries
!pip install ollama gradio

# Start Ollama in a separate process
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

running = subprocess.run(["pgrep", "-a", "ollama"], capture_output=True, text=True)
print("Ollama Status:", running.stdout if running.stdout else "Ollama is NOT running!")

# Create symlink
!rm -rf ~/.ollama/models
!ln -s /content/drive/MyDrive/ollama_models ~/.ollama/models


### Download the model to Google Drive

To change the model:
1. Find a model at [Ollama](https://ollama.com/search).

  Use a small model because you don't want to exceed the storage space in your Google Drive storage space and the compute resources in Google Colab.

2. Modify this line and replace `gemma3:1b` with the new model:

  ```bash
  !ollama pull gemma3:1b
  ```

3. After the model is downloaded, go to Google Drive and note the path of the model.

  Update the model path. This ensures the model won't be redownloaded the next time you run this notebook.

  ```bash
  model_info_path = '/content/drive/MyDrive/ollama_models/manifests/registry.ollama.ai/library/gemini3/1b'
  ```


In [ ]:
import os
model_info_path = '/content/drive/MyDrive/ollama_models/manifests/registry.ollama.ai/library/gemma3/1b'
if not os.path.exists(model_info_path):
    !ollama pull gemma3:1b
else:
    print("Model already exists. Skipping download.")

### Define the chatbot function

In [ ]:
context = [] #Initialize outside of the Blocks

def generate(prompt, context, model, temperature, num_predict):
    """Call the Ollama API to generate a response."""
    api_response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            'model': model,
            'prompt': prompt,
            'context': context,
            'options': {
                'temperature': temperature,
                'num_predict': num_predict
            }
        },
        stream=False
    )
    api_response.raise_for_status()

    response = ""
    for line in api_response.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        if 'error' in body:
            raise Exception(body['error'])
        response += response_part
        if body.get('done', False):
            context = body.get('context', [])
    return response, context

def chatbot_response(user_message, chat_history, user_prompt, temperature, max_tokens):
    """Generate chatbot responses dynamically."""
    global context

    # Format conversation history and prompt template with fixed structure
    chat_history_formatted = "\n".join([f"{role}: {msg}" for role, msg in chat_history])

    # Fixed prompt template with user-defined prompt at the top
    prompt_template = f"""{user_prompt}
Conversation History:
{chat_history_formatted}
User's Message: {user_message}
Chatbot's Response:"""

    try:
        # Generate response using Ollama API
        response, context = generate(prompt_template, context, model="gemma3:1b", temperature=temperature, num_predict=max_tokens)
        bot_message = response.strip()
    except Exception as e:
        bot_message = f"Error: {e}"

    # Update chat history
    chat_history.append(("User", user_message))
    chat_history.append(("Chatbot", bot_message))

    return chat_history, chat_history

### Define the chatbot interface

In [ ]:
# Define Gradio interface with fixed dimensions using CSS styling
DEFAULT_USER_PROMPT = "You are a helpful assistant."

with gr.Blocks(css=".container {width: 1280px; height: 350px;} .compact-textbox {padding: 0; margin: 0; height: 150px;} .chatbot {max-height: 200px; overflow-y: auto;} .message-box textarea {height: 30px !important; min-height: 30px !important; max-height: 30px !important;}") as demo:
    gr.Markdown("# 🤖 Chatbot (Google Colab)")

    # Initialize chat history and context
    chat_history = gr.State([])
    context = []  # Reset context for memory

    def chatbot_response(user_message, chat_history, user_prompt, temperature, max_tokens):
        """Generate chatbot responses dynamically."""
        global context

        # Format conversation history and prompt template with fixed structure
        chat_history_formatted = "\n".join([f"{role}: {msg}" for role, msg in chat_history])

        # Fixed prompt template with user-defined prompt at the top
        prompt_template = f"""{user_prompt}
Conversation History:
{chat_history_formatted}
User's Message: {user_message}
Chatbot's Response:"""

        try:
            # Generate response using Ollama API
            response, context = generate(prompt_template, context, model="gemma3:1b", temperature=temperature, num_predict=max_tokens)
            bot_message = response.strip()
        except Exception as e:
            bot_message = f"Error: {e}"

        # Update chat history
        chat_history.append(("User", user_message))
        chat_history.append(("Chatbot", bot_message))

        return chat_history, chat_history

    def clear_chat():
        """Clear the chat history and reset context."""
        global context
        context = []  # Reset context for memory
        return [], []  # Clear chat history

    with gr.Row():
        # Left Column for Prompt and Sliders
        with gr.Column(scale=1):
            user_prompt_input = gr.Textbox(
                value=DEFAULT_USER_PROMPT,
                lines=5,
                max_lines=5,
                label=None,
                info="Modify this prompt to change the chatbot's persona.",
                elem_classes=["compact-textbox"]
            )

            temperature_slider = gr.Slider(
                minimum=0.0,
                maximum=1.0,
                step=0.1,
                value=0.7,
                label="Temperature",
                info="Adjust the randomness of the chatbot's responses."
            )

            max_tokens_slider = gr.Slider(
                minimum=50,
                maximum=500,
                step=50,
                value=200,
                label="Max Tokens",
                info="Set the maximum length of the chatbot's response."
            )

        # Right Column for Chatbox and Message Input
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(elem_classes=["chatbot"])
            msg_input = gr.Textbox(
                label="Your Message",
                placeholder="Ask me anything!",
                lines=1,  # Display only one line
                max_lines=1,  # Prevent vertical expansion
                elem_classes=["message-box"]
            )
            send_btn = gr.Button("Send")
            clear_btn = gr.Button("Clear")  # Clear button

    send_btn.click(
        chatbot_response,
        inputs=[msg_input, chat_history, user_prompt_input, temperature_slider, max_tokens_slider],
        outputs=[chatbot, chat_history]
    )

    clear_btn.click(
        clear_chat,
        inputs=[],
        outputs=[chatbot, chat_history]  # Clear both chatbot display and history
    )

demo.launch(share=True)
